<a href="https://colab.research.google.com/github/bahram3000/test_balancer_app/blob/main/start_deposit_sawp_remove_testnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq web3

In [26]:
weth_usdc_pool_addres='0x6418eec70f50913ff0d756b48d32ce7c02b47c47'
eth_usdc_pool_addres='0x3289680dD4d6C10bb19b899729cda5eEF58AEfF1'

In [3]:
from web3 import Web3
from eth_account import Account

In [41]:
import requests
import json
import yfinance as yf
import time

In [59]:
RPC = "https://eth-sepolia.g.alchemy.com/v2/demo"
w3 = Web3(Web3.HTTPProvider(RPC))

# کلید خصوصی والت تست‌نت
PRIVATE_KEY = "75074ee5760d9640d993d09bc6e969b37cadcc2c83195886d43b3e86b2b8b6da"
account = Account.from_key(PRIVATE_KEY)
address = account.address

print("Connected:", w3.is_connected())
print("Your address:", address)


Connected: True
Your address: 0x7868EDEfC36A74eba220C75aCFB05c59a4e55822


In [7]:
eth_balance = w3.eth.get_balance(address)
print("ETH Balance:", w3.from_wei(eth_balance, 'ether'))

ETH Balance: 0.055


In [11]:

etherscan_apikey = "N68HP1FGB4UQQFBHZQUJIM12HVYWJME96M"
ADDRESS = "0x7868EDEfC36A74eba220C75aCFB05c59a4e55822"
CHAIN_ID = 11155111  # Sepolia

# ۱) موجودی ETH
eth_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=balance"
    f"&address={ADDRESS}&tag=latest&apikey={etherscan_apikey}"
)

eth_res = requests.get(eth_url).json()
print("ETH balance:", int(eth_res.get("result", 0))/10**18)

# ۲) درخواست لیست توکن — **دقت**: تا جای من اطلاع، Etherscan V2 فعلاً endpoint عمومی برای list token ندارد
tokens_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=tokenlist"
    f"&address={ADDRESS}&apikey={etherscan_apikey}"
)

tokens_res = requests.get(tokens_url).json()
print("Tokens response:", tokens_res)


ETH balance: 0.055
Tokens response: {'status': '0', 'message': 'NOTOK', 'result': 'Error! Missing Or invalid Action name'}


In [12]:
ERC20_ABI = '''
[
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
]
'''

TOKEN_LIST = [
    "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",
    "0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238",
]

for token in TOKEN_LIST:
    contract = w3.eth.contract(address=token, abi=ERC20_ABI)
    bal = contract.functions.balanceOf(ADDRESS).call()
    dec = contract.functions.decimals().call()
    sym = contract.functions.symbol().call()
    print(sym, bal / (10 ** dec))


WETH 0.001
USDC 0.0


In [36]:
# آدرس رسمی WETH روی Sepolia
WETH = Web3.to_checksum_address("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14")

# ABI اصلی توابع deposit/withdraw
WETH_ABI = json.loads("""
[
    {
        "constant": false,
        "inputs": [],
        "name": "deposit",
        "outputs": [],
        "payable": true,
        "stateMutability": "payable",
        "type": "function"
    },
    {
        "constant": false,
        "inputs": [{"name":"wad","type":"uint256"}],
        "name": "withdraw",
        "outputs": [],
        "payable": false,
        "stateMutability": "nonpayable",
        "type": "function"
    }
]
""")

contract = w3.eth.contract(address=WETH, abi=WETH_ABI)

# ------------------------------
# مقدار ETH که می‌خواهی Wrap کنی
# ------------------------------
amount_eth = 0.001
amount_wei = w3.to_wei(amount_eth, "ether")

# ------------------------------
# ساخت تراکنش wrap → deposit()
# ------------------------------

tx = contract.functions.deposit().build_transaction({
    "from": address,
    "value": amount_wei,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111,
})

estimated_gas = w3.eth.estimate_gas(tx)
print("Estimated gas:", estimated_gas)




estimated_gas = w3.eth.estimate_gas(tx)
print("Estimated gas:", estimated_gas)

# gas_used که گرفتیم
gas_used = estimated_gas

eth_price = requests.get("https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd").json()["ethereum"]["usd"]

# gas_price شبکه (Wei)
gas_price = w3.eth.gas_price  # مثال: 5 gwei = 5_000_000_000

# سه حالت
gas_price_slow   = int(gas_price * 0.7)
gas_price_normal = gas_price
gas_price_fast   = int(gas_price * 1.3)

cost_slow   = gas_used * gas_price_slow * eth_price / 1e18
cost_normal = gas_used * gas_price_normal * eth_price / 1e18
cost_fast   = gas_used * gas_price_fast * eth_price / 1e18

print("Slow cost  USD:", cost_slow)
print("Normal cost USD:", cost_normal)
print("Fast cost  USD:", cost_fast)

Estimated gas: 28284
Estimated gas: 28284
Slow cost  USD: 6.002520988800001e-05
Normal cost USD: 8.575029984e-05
Fast cost  USD: 0.00011147538979200001


In [31]:
address

'0x7868EDEfC36A74eba220C75aCFB05c59a4e55822'

In [37]:
tx = contract.functions.deposit().build_transaction({
    "from": address,
    "value": amount_wei,
    "gas": 150000,               # safe limit
    "gasPrice": gas_price_slow,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111          # chainId سپولیا
})


# فرض RPC و tx ساخته شده است
signed_txn = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)

# ارسال تراکنش
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)

print("TX sent:", tx_hash.hex())
print("Track here: https://sepolia.etherscan.io/tx/" +'0x' +tx_hash.hex())

TX sent: 17adce6013d93e0d124785977f0ecbec735f0c9563a7539ebbc7828bfc44b45d
Track here: https://sepolia.etherscan.io/tx/0x17adce6013d93e0d124785977f0ecbec735f0c9563a7539ebbc7828bfc44b45d


In [49]:
def check_token_balance(token_list_address:list, inp_address):
  ERC20_ABI = '''
  [
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],
   "name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],
   "type":"function"},
  {"constant":true,"inputs":[],"name":"decimals",
   "outputs":[{"name":"","type":"uint8"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"symbol",
   "outputs":[{"name":"","type":"string"}],"type":"function"}
  ]
  '''
  balances = []
  for token in token_list_address:
      contract = w3.eth.contract(address=token, abi=ERC20_ABI)
      bal = contract.functions.balanceOf(inp_address).call()
      dec = contract.functions.decimals().call()
      sym = contract.functions.symbol().call()
      balances.append([sym, bal / (10 ** dec)])
  return balances

In [47]:
TOKEN_LIST = [
    "0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14",
    "0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238"
]

In [50]:
check_token_balance(TOKEN_LIST, address)

[['WETH', 0.002], ['USDC', 0.0]]

In [39]:
# ۱) موجودی ETH
eth_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=balance"
    f"&address={ADDRESS}&tag=latest&apikey={etherscan_apikey}"
)

eth_res = requests.get(eth_url).json()
print("ETH balance:", int(eth_res.get("result", 0))/10**18)

# ۲) درخواست لیست توکن — **دقت**: تا جای من اطلاع، Etherscan V2 فعلاً endpoint عمومی برای list token ندارد
tokens_url = (
    f"https://api.etherscan.io/v2/api?"
    f"chainid={CHAIN_ID}&module=account&action=tokenlist"
    f"&address={ADDRESS}&apikey={etherscan_apikey}"
)

tokens_res = requests.get(tokens_url).json()
print("Tokens response:", tokens_res)


ETH balance: 0.032954791404638896
Tokens response: {'status': '0', 'message': 'NOTOK', 'result': 'Error! Missing Or invalid Action name'}


In [40]:
pool = w3.eth.contract(
    address=eth_usdc_pool_addres,
    abi=[
        {"constant":True,"inputs":[],"name":"token0","outputs":[{"name":"","type":"address"}],"type":"function"},
        {"constant":True,"inputs":[],"name":"token1","outputs":[{"name":"","type":"address"}],"type":"function"}
    ]
)

try:
    t0 = pool.functions.token0().call()
    t1 = pool.functions.token1().call()
    print("Pool tokens:", t0, t1)
except Exception as e:
    print("Not a valid pool or cannot read:", e)


Pool tokens: 0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238 0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14


In [42]:
# -------- آدرس‌ها
WETH = Web3.to_checksum_address("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14")
USDC = Web3.to_checksum_address("0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238")
ROUTER = Web3.to_checksum_address("0xE592427A0AEce92De3Edee1F18E0157C05861564")  # Uniswap v3 Router

# -------- ABI Router v3: exactInputSingle
router_abi = json.loads("""
[
  {
    "inputs": [
      {
        "components": [
          {"internalType": "address","name": "tokenIn","type": "address"},
          {"internalType": "address","name": "tokenOut","type": "address"},
          {"internalType": "uint24","name": "fee","type": "uint24"},
          {"internalType": "address","name": "recipient","type": "address"},
          {"internalType": "uint256","name": "deadline","type": "uint256"},
          {"internalType": "uint256","name": "amountIn","type": "uint256"},
          {"internalType": "uint256","name": "amountOutMinimum","type": "uint256"},
          {"internalType": "uint160","name": "sqrtPriceLimitX96","type": "uint160"}
        ],
        "internalType": "struct ISwapRouter.ExactInputSingleParams",
        "name": "params",
        "type": "tuple"
      }
    ],
    "name": "exactInputSingle",
    "outputs": [{"internalType": "uint256","name": "amountOut","type": "uint256"}],
    "stateMutability": "payable",
    "type": "function"
  }
]
""")

router = w3.eth.contract(address=ROUTER, abi=router_abi)

# -------- پارامترهای تراکنش
amount_in_eth = 0.002
amount_in_wei = w3.to_wei(amount_in_eth, "ether")

fee = 500
recipient = address
deadline = int(time.time()) + 600  # 10 دقیقه

amount_out_min = 1  # حداقل USDC دریافتی (slippage باید محاسبه شود)
sqrtPriceLimitX96 = 0  # بدون محدودیت قیمت

params = (
    WETH,
    USDC,
    fee,
    recipient,
    deadline,
    amount_in_wei,
    amount_out_min,
    sqrtPriceLimitX96
)

# -------- Gas
gas_price = int(w3.eth.gas_price * 0.7)  # slow
gas_limit = 250000

# -------- ساخت تراکنش
tx = router.functions.exactInputSingle(params).build_transaction({
    "from": address,
    "value": amount_in_wei,
    "gas": gas_limit,
    "gasPrice": gas_price,
    "nonce": w3.eth.get_transaction_count(address),
    "chainId": 11155111
})

# -------- امضا و ارسال
signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)

print("TX sent:", tx_hash.hex())
print("Track here: https://sepolia.etherscan.io/tx/" +'0x'+ tx_hash.hex())


TX sent: 3d7afa45540363d156ad085980c72b098060d15219c5fb2437a4da046c4e086e
Track here: https://sepolia.etherscan.io/tx/0x3d7afa45540363d156ad085980c72b098060d15219c5fb2437a4da046c4e086e


In [56]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.002], ['USDC', 20.242914]]

In [52]:
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction status:", receipt.status)

Transaction status: 1


In [53]:
erc20_abi = json.loads("""
[
  {"constant":true,"inputs":[{"name":"owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"type":"function"},
  {"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"}
]
""")

usdc_contract = w3.eth.contract(address=USDC, abi=erc20_abi)
balance = usdc_contract.functions.balanceOf(address).call()
decimals = usdc_contract.functions.decimals().call()
print("USDC balance:", balance / (10**decimals))

USDC balance: 0.0


In [61]:
# -------- آدرس‌ها
WETH = Web3.to_checksum_address("0xfFf9976782d46CC05630D1f6eBAb18b2324d6B14")
USDC = Web3.to_checksum_address("0x1c7D4B196Cb0C7B01d743Fbc6116a902379C7238")
ROUTER = Web3.to_checksum_address("0xE592427A0AEce92De3Edee1F18E0157C05861564")

# -------- ABI Router v3 (فقط تابع exactInputSingle)
router_abi = json.loads("""
[
  {
    "inputs": [
      {
        "components": [
          {"internalType": "address","name": "tokenIn","type": "address"},
          {"internalType": "address","name": "tokenOut","type": "address"},
          {"internalType": "uint24","name": "fee","type": "uint24"},
          {"internalType": "address","name": "recipient","type": "address"},
          {"internalType": "uint256","name": "deadline","type": "uint256"},
          {"internalType": "uint256","name": "amountIn","type": "uint256"},
          {"internalType": "uint256","name": "amountOutMinimum","type": "uint256"},
          {"internalType": "uint160","name": "sqrtPriceLimitX96","type": "uint160"}
        ],
        "internalType": "struct ISwapRouter.ExactInputSingleParams",
        "name": "params",
        "type": "tuple"
      }
    ],
    "name": "exactInputSingle",
    "outputs": [{"internalType": "uint256","name": "amountOut","type": "uint256"}],
    "stateMutability": "payable",
    "type": "function"
  }
]
""")

# -------- ERC20 ABI ساده برای approve و balance
erc20_abi = json.loads("""
[
  {"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"type":"function"},
  {"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"success","type":"bool"}],"type":"function"}
]
""")

# -------- کانترکت‌ها
router = w3.eth.contract(address=ROUTER, abi=router_abi)
weth = w3.eth.contract(address=WETH, abi=erc20_abi)
usdc = w3.eth.contract(address=USDC, abi=erc20_abi)

# -------- مقدار ورودی
amount_in_eth = 0.002
amount_in_wei = w3.to_wei(amount_in_eth, "ether")

# -------- approve WETH (در صورت استفاده از WETH)
nonce = w3.eth.get_transaction_count(address)
approve_tx = weth.functions.approve(ROUTER, amount_in_wei).build_transaction({
    "from": address,
    "gas": 100000,
    "gasPrice": int(w3.eth.gas_price * 1.0),
    "nonce": nonce
})
signed_approve = w3.eth.account.sign_transaction(approve_tx, PRIVATE_KEY)
approve_hash = w3.eth.send_raw_transaction(signed_approve.raw_transaction)
w3.eth.wait_for_transaction_receipt(approve_hash)
print("Approved WETH for Router:", approve_hash.hex())

# -------- پارامترهای swap
fee = 500
recipient = address
deadline = int(time.time()) + 600
amount_out_min = 0  # می‌توان slippage واقعی را محاسبه کرد
sqrtPriceLimitX96 = 0

params = (
    WETH,
    USDC,
    fee,
    recipient,
    deadline,
    amount_in_wei,
    amount_out_min,
    sqrtPriceLimitX96
)

# -------- Gas
gas_price = int(w3.eth.gas_price * 0.7)
gas_limit = 250000

# -------- موجودی USDC قبل
usdc_before = usdc.functions.balanceOf(address).call()

# -------- ساخت تراکنش swap
nonce = w3.eth.get_transaction_count(address)
tx = router.functions.exactInputSingle(params).build_transaction({
    "from": address,
    "value": 0,  # چون WETH است
    "gas": gas_limit,
    "gasPrice": gas_price,
    "nonce": nonce,
    "chainId": 11155111
})

signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
print("Swap TX sent:", tx_hash.hex())

# -------- انتظار تا mined شدن تراکنش
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("TX mined:", receipt.transactionHash.hex())

# -------- موجودی USDC بعد و مقدار واقعی دریافتی
usdc_after = usdc.functions.balanceOf(address).call()
print("USDC received:", usdc_after - usdc_before)


Approved WETH for Router: aeb18ad571b2fc437a9a37f5cfce19ffa56907bb8e97f9482b790bca2c10210d
Swap TX sent: 61c9413c9526c0cc4607b527275fb95da11f57c6af945620c6a2f278e420260c
TX mined: 61c9413c9526c0cc4607b527275fb95da11f57c6af945620c6a2f278e420260c
USDC received: 0


In [66]:
eth_balance = w3.eth.get_balance(address)
print("ETH Balance:", w3.from_wei(eth_balance, 'ether'))

ETH Balance: 0.081737706151980272


In [67]:
check_token_balance(TOKEN_LIST,address)

[['WETH', 0.002], ['USDC', 20.242914]]